In [1]:
import warnings
import seaborn as sns
import matplotlib.pylab as plt
import PIL
import pandas as pd
import os
from PIL import Image

import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception, VGG16, ResNet50, InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator


import numpy as np # linear algebra

from numpy.random import seed

Using TensorFlow backend.


In [2]:
K.clear_session()

In [3]:
SEED = 2
seed(SEED)
tf.set_random_seed(SEED)

In [4]:
DATA_PATH = '../input'
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')
TRAIN_CROP_PATH = os.path.join(DATA_PATH, 'train_crop_299')
TEST_CROP_PATH = os.path.join(DATA_PATH, 'test_crop_299')

df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))
image_size=299

In [5]:
#https://github.com/yu4u/cutout-random-erasing
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        input_img = preprocess_input(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [6]:

class MixupImageDataGenerator():
    def __init__(self, generator, dataframe, directory, target, batch_size, img_size, alpha=0.2, subset=None, scale='rgb'):
        """Constructor for mixup image data generator.

        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.

        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha
        # First iterator yielding tuples of (x, y)
        self.generator1 = generator.flow_from_dataframe(dataframe=dataframe, 
                                                        directory=directory,
                                                        x_col = 'img_file',
                                                        y_col = target,
                                                        target_size=(image_size, image_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        seed=3,
                                                        color_mode=scale,
                                                        shuffle=True,)

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_dataframe(dataframe=dataframe, 
                                                        directory=directory,
                                                        x_col = 'img_file',
                                                        y_col = target,
                                                        target_size=(image_size, image_size),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        seed=5,
                                                        color_mode=scale,
                                                        shuffle=True)


        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.

        Returns:
            int -- steps per epoch.
        """
        if (num_samples % batch_size) > 0:
            return (num_samples // batch_size) + 1
        else:
            return num_samples // batch_size

    def __next__(self):
        """Get next batch input/output pair.

        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0
        
        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()
        size = X1.shape[0]
        # random sample the lambda value from beta distribution.
        l = np.random.randint(90, 100, size=size) / 100
        X_l = l.reshape(size, 1, 1, 1)
        y_l = l.reshape(size, 1)

        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)


In [7]:
def get_generator(train_df, val_df, train_dir, valid_dir, test_df, test_dir, image_size, batch_size,valid_batch_size,
                 scale='rgb', target='class'):
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df, 
        directory=train_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
        shuffle=True,
        #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True)
    )

    validation_generator = valid_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=valid_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size,image_size),
        batch_size=valid_batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale
        #processing_function=preprocess_input
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_dir,
        x_col='img_file',
        y_col=None,
        target_size= (image_size,image_size),
        color_mode=scale,
        class_mode=None,
        batch_size=batch_size,
        shuffle=False,
        #preprocessing_function=preprocess_input
    )
    return train_generator, validation_generator, test_generator

In [8]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.1
    )
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [9]:
df_train["class"] = df_train["class"].astype('str')
df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

In [10]:
def split_traindf(df, train_size=0.6, stratify=True, label='class'):
    target = None
    if stratify:
        target = df[label].values
    X_train, X_val = train_test_split(df, train_size=train_size, random_state=SEED, stratify=target)
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    return X_train, X_val

In [11]:
X_train, X_val = split_traindf(df_train.iloc[:, :], train_size=0.8, stratify=True)
nb_train_sample = X_train.shape[0]
nb_validation_sample = X_val.shape[0]
nb_test_sample = df_test.shape[0]
scale = 'rgb'

In [12]:
batch_size=16
valid_batch_size=16

In [13]:
train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                    val_df=X_val,
                                                    train_dir=TRAIN_CROP_PATH,
                                                    valid_dir=TRAIN_CROP_PATH,
                                                    test_df=df_test,
                                                    test_dir=TEST_CROP_PATH,
                                                    image_size=image_size,
                                                    batch_size=batch_size,
                                                    scale=scale,
                                                    valid_batch_size=valid_batch_size)

Found 8012 validated image filenames belonging to 196 classes.
Found 2004 validated image filenames belonging to 196 classes.
Found 6169 validated image filenames.


In [14]:
train_gen = MixupImageDataGenerator(generator = train_datagen,
                                    dataframe = X_train,
                                    directory=TRAIN_CROP_PATH,
                                    batch_size=batch_size,
                                    img_size=image_size,
                                    target='class')

Found 8012 validated image filenames belonging to 196 classes.
Found 8012 validated image filenames belonging to 196 classes.


In [15]:
def get_model(app, image_size, opt, num_class=196, lr=0.0001):
    if app=='Xception':
        application = Xception
    elif app=='VGG16':
        application = VGG16
    elif app=='ResNet50':
        application = ResNet50
    elif app=='InceptionV3':
        application = InceptionV3
    base_model = application(weights='imagenet', input_shape=(image_size,image_size,3), include_top=False)
    #base_model.trainable = False
    

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(512, activation='sigmoid'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(num_class, activation='softmax'))
    #model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])

    return model

In [16]:
def get_model_path(model_dir, model_name, model_type):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    from datetime import datetime
    now = datetime.now()
    date_time = now.strftime("%m%d_%H%M")
    model_path =model_dir +  "{}_{}_{}.{}".format(model_name , date_time ,model_type, '.hdf5')
    print('>>model path to save: {}'.format(model_path))
    return model_path

In [17]:
model_type='Xception'
image_size = 299 if model_type=='Xception' else 224
histories=[]
patient = 10
lr = 0.0001
epoch=300

In [18]:
model_name = 'baseline_mixup'
model_dir = '../model/'
model_path = get_model_path(model_dir, model_name, model_type)
model = get_model(app=model_type, image_size=image_size, opt=optimizers.RMSprop(lr=lr), lr=lr)
print('>>get model completed')

>>model path to save: ../model/baseline_mixup_0708_0704_Xception..hdf5
>>get model completed


In [24]:
from keras.callbacks import Callback

class printLearningRate(Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        decay = self.model.optimizer.decay
        iterations = self.model.optimizer.iterations
        lr_with_decay = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        print('Current LR : ', K.eval(lr_with_decay))

In [25]:
def get_callback(patient, model_path, lr, total_count):
    callbacks = [
        EarlyStopping(monitor='val_loss',
                      patience=patient,
                      mode='min',
                      verbose=1),
        ModelCheckpoint(filepath=model_path,
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True,
                        mode='min'),
        ReduceLROnPlateau(monitor = 'val_loss',
                          patience = 1,
                          min_lr=lr * 0.0001,
                          verbose=True,
                          mode='min'),
        printLearningRate()
    ]
    return callbacks

In [26]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    

In [27]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=get_steps(nb_train_sample, batch_size),
    epochs=epoch,
    validation_data=validation_gen,
    validation_steps=get_steps(nb_validation_sample, valid_batch_size),
    verbose=1,
    callbacks=get_callback(patient, model_path, lr, len(X_train))
)

Epoch 1/300
501/501 [==============================] - 201s 401ms/step - loss: 3.3749 - acc: 0.3523 - val_loss: 2.3352 - val_acc: 0.5374

Epoch 00001: val_loss improved from inf to 2.33516, saving model to ../model/baseline_mixup_0708_0704_Xception..hdf5
Current LR :  1e-04
Epoch 2/300
501/501 [==============================] - 201s 401ms/step - loss: 2.3922 - acc: 0.5850 - val_loss: 1.5205 - val_acc: 0.7131

Epoch 00002: val_loss improved from 2.33516 to 1.52048, saving model to ../model/baseline_mixup_0708_0704_Xception..hdf5
Current LR :  1e-04
Epoch 3/300
501/501 [==============================] - 201s 401ms/step - loss: 1.6856 - acc: 0.7359 - val_loss: 0.9667 - val_acc: 0.8034

Epoch 00003: val_loss improved from 1.52048 to 0.96674, saving model to ../model/baseline_mixup_0708_0704_Xception..hdf5
Current LR :  1e-04
Epoch 4/300
501/501 [==============================] - 201s 401ms/step - loss: 1.2295 - acc: 0.8329 - val_loss: 0.6828 - val_acc: 0.8413

Epoch 00004: val_loss improve

KeyboardInterrupt: 

use 5000 images
baseline + ratio image test
